## Tugas: Implementasi Naive Bayes Kategorial untuk Prediksi Cuaca Besok
### Nama: Rahmat Mulia
### NIM: 235520110141
### Unit: A (2023)

#### Deskripsi Singkat
Pada tugas ini saya mengimplementasikan algoritma Naive Bayes untuk memprediksi apakah besok akan hujan berdasarkan data kategorial cuaca hari ini. Perhitungan dilakukan secara manual mulai dari menghitung prior, likelihood (menggunakan Laplace smoothing), skor kelas, posterior, hingga menghasilkan label prediksi untuk beberapa contoh kasus cuaca.

In [139]:
import pandas as pd
import numpy as np


In [140]:
df = pd.read_excel("dataset_hujan_besok_kategori.xlsx")
df.head()


,Temp,Humidity,Wind,Cloud,Pressure,RainToday,RainTomorrow
0,panas,lembab,tenang,cerah,rendah,No,No
1,panas,lembab,pelan,mendung,rendah,Yes,Yes
2,panas,kering,kencang,cerah,tinggi,No,No
3,dingin,lembab,tenang,mendung,rendah,No,No
4,dingin,kering,pelan,cerah,tinggi,Yes,No


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Temp          20 non-null     object
 1   Humidity      20 non-null     object
 2   Wind          20 non-null     object
 3   Cloud         20 non-null     object
 4   Pressure      20 non-null     object
 5   RainToday     20 non-null     object
 6   RainTomorrow  20 non-null     object
dtypes: object(7)
memory usage: 1.2+ KB


### Penjelasan Dataset

Dataset ini berisi data kondisi cuaca harian dengan fitur-fitur yang bersifat kategorial. Setiap fitur menggambarkan keadaan cuaca pada hari ini, sedangkan kolom *RainTomorrow* menunjukkan apakah besok akan terjadi hujan atau tidak. Adapun penjelasan masing-masing fitur adalah sebagai berikut:

- **Temp** — kondisi suhu udara: *panas*, *sedang*, atau *dingin*.  
- **Humidity** — tingkat kelembaban: *lembab* atau *kering*.  
- **Wind** — kondisi angin: *tenang*, *pelan*, atau *kencang*.  
- **Cloud** — kondisi awan: *cerah* atau *mendung*.  
- **Pressure** — tekanan udara: *rendah* atau *tinggi*.  
- **RainToday** — apakah hari ini hujan: *Yes* atau *No*.  
- **RainTomorrow** — target prediksi, menunjukkan apakah besok hujan: *Yes* atau *No*.

Pada tugas ini, kolom **RainTomorrow** digunakan sebagai variabel target untuk proses klasifikasi menggunakan algoritma Naive Bayes.


In [142]:
yes = len(df[df["RainTomorrow"] == "Yes"])
no  = len(df[df["RainTomorrow"] == "No"])
N = len(df)

P_yes = yes / N
P_no = no / N

print("Jumlah Yes:", yes)
print("Jumlah No :", no)
print("Total Data:", N)
print("P(Yes) =", P_yes)
print("P(No)  =", P_no)


Jumlah Yes: 8
Jumlah No : 12
Total Data: 20
P(Yes) = 0.4
P(No)  = 0.6


### Perhitungan Prior
Prior menggambarkan peluang awal terjadinya hujan atau tidak hujan tanpa mempertimbangkan fitur cuaca lainnya.

- P(Yes) = jumlah data dengan RainTomorrow = Yes dibagi total data  
- P(No)  = jumlah data dengan RainTomorrow = No dibagi total data  

Prior digunakan sebagai dasar dalam perhitungan skor posterior pada Naive Bayes.


In [143]:
def hitung_kategori_unik(df, kolom):
    return df[kolom].unique()

def hitung_p_val_kelas(df, feature, value, kelas, target="RainTomorrow"):
    K = df[feature].nunique()
    count_val_dan_kelas = len(df[(df[feature] == value) & (df[target] == kelas)])
    count_kelas = len(df[df[target] == kelas])
    return (count_val_dan_kelas + 1) / (count_kelas + K)


In [144]:
print("P(Temp='panas' | Yes) =", hitung_p_val_kelas(df, "Temp", "panas", "Yes"))
print("P(Humidity='lembab' | No) =", hitung_p_val_kelas(df, "Humidity", "lembab", "No"))


P(Temp='panas' | Yes) = 0.45454545454545453
P(Humidity='lembab' | No) = 0.35714285714285715


### Perhitungan Likelihood
Likelihood dihitung menggunakan Laplace smoothing:

P(val | kelas) = (count(feature = value dan target = kelas) + 1) / (count(target = kelas) + K)

Metode smoothing ini digunakan untuk menghindari probabilitas 0 ketika sebuah kategori tidak muncul dalam kelas tertentu.


In [145]:
def predict_naive_bayes(x):
    classes = ["Yes", "No"]
    priors = {"Yes": P_yes, "No": P_no}
    scores = {}

    for c in classes:
        product = 1
        for f in x:
            p = hitung_p_val_kelas(df, f, x[f], c)
            product *= p
        scores[c] = priors[c] * product
    
    P_yes_post = scores["Yes"] / (scores["Yes"] + scores["No"])
    P_no_post  = scores["No"]  / (scores["Yes"] + scores["No"])

    pred = "Yes" if P_yes_post > P_no_post else "No"
    return pred, P_yes_post, P_no_post


In [146]:
case1 = {
    "Temp": "panas",
    "Humidity": "lembab",
    "Wind": "tenang",
    "Cloud": "mendung",
    "Pressure": "rendah",
    "RainToday": "Yes"
}
predict_naive_bayes(case1)


('Yes', 0.9754160098745559, 0.024583990125444172)

In [147]:
case2 = {
    "Temp": "sedang",
    "Humidity": "kering",
    "Wind": "kencang",
    "Cloud": "cerah",
    "Pressure": "tinggi",
    "RainToday": "No"
}
predict_naive_bayes(case2)


('No', 0.002037299292927322, 0.9979627007070727)

In [148]:
case3 = {
    "Temp": "dingin",
    "Humidity": "lembab",
    "Wind": "pelan",
    "Cloud": "mendung",
    "Pressure": "rendah",
    "RainToday": "Yes"
}
predict_naive_bayes(case3)


('Yes', 0.952011762115404, 0.047988237884595976)

### Interpretasi Hasil Prediksi

**Case 1**  
**Kondisi cuaca:**  
Temp = panas, Humidity = lembab, Wind = tenang, Cloud = mendung, Pressure = rendah, RainToday = Yes  

**Hasil:**  
Model memprediksi **RainTomorrow = Yes**  
- P(Yes) = **0.9754**  
- P(No)  = **0.0246**  

Artinya, terdapat probabilitas sekitar **97.5%** bahwa besok akan hujan.

---

**Case 2**  
**Kondisi cuaca:**  
Temp = sedang, Humidity = kering, Wind = kencang, Cloud = cerah, Pressure = tinggi, RainToday = No  

**Hasil:**  
Model memprediksi **RainTomorrow = No**  
- P(Yes) = **0.0020**  
- P(No)  = **0.9980**  

Artinya, terdapat probabilitas sekitar **99.8%** bahwa besok **tidak** akan hujan.

---

**Case 3**  
**Kondisi cuaca:**  
Temp = dingin, Humidity = lembab, Wind = pelan, Cloud = mendung, Pressure = rendah, RainToday = Yes  

**Hasil:**  
Model memprediksi **RainTomorrow = Yes**  
- P(Yes) = **0.9520**  
- P(No)  = **0.0480**  

Artinya, terdapat probabilitas sekitar **95.2%** bahwa besok akan hujan.
